<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 13](https://adventofcode.com/2021/day/13): Transparent Origami
### Part I
- **Unknown**: The number of dots visible after completing just the first fold instruction on the transparent paper.
- **Data**: Coordinates of dots and folding instructions
- **Condition**:
  - [0,0] represents the top-left coordinates
  - `x` increases to the right, `y` increases downward
  - `fold along y=...` instruction means folding the paper up
  - `fold along x=xxx` instruction means folding the paper left
- **Plan**:
  - Initialize the matrix
  - Write a function for folding
  - Count the dots after folding


In [2]:
example = """6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5"""

In [57]:
def parse(input):
  lines = input.split('\n')
  dots = []
  for line in lines:
    if ',' in line:
      dots.append(np.array(line.split(',')).astype(int))
  return np.array(dots)

dots = parse(example)
dots[:4]

array([[ 6, 10],
       [ 0, 14],
       [ 9, 10],
       [ 0,  3]])

In [60]:
pap = np.zeros(dots.max(axis=0)+1).astype(int)
for dot in dots:
  pap[dot[0], dot[1]] = 1
pap

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

**Beware** of slicing vs. indexing. The expression `pap[6,10]` adresses one element [6,10] in the array, whereas the expression `pap[[6,10]]` addresses two rows [6, :] and [10, :]

In [71]:
print(dots[0])
print(pap[dots[0]])
print(np.stack((pap[6, :], pap[10, :])))

[ 6 10]
[[1 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]]
[[1 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]]


array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True]])

In [46]:
# Fold the paper up - indices
y = 7
for i in range(y):
  print(i, 2*y-i)

0 14
1 13
2 12
3 11
4 10
5 9
6 8


In [72]:
pap_y7 = pap[:,:y].copy()
for i in range(y):
  pap_y7[:,i] += pap[:, 2*y-i]
pap_y7.T

array([[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [73]:
pap_y7[pap_y7>0].shape

(17,)

In [75]:
# Fold the paper left
x = 5
pap_x5 = pap_y7[:x,:].copy()
for i in range(x):
  pap_x5[i,:] += pap_y7[2*x-i, :]
pap_x5.T

array([[1, 1, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 2, 2, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [77]:
pap_x5[pap_x5>0].shape

(16,)

### Part II
- **Unknown**: 
- **Data**: 
- **Condition**:
  - 
  - 
- **Plan**:
  - 
